In [1]:
from pyspark.sql import SparkSession
import altair as alt
import pandas as pd

spark = SparkSession.builder.getOrCreate()

In [2]:
def load(name):
    print(f"===================== {name} =====================")
    df = spark.read.parquet(f"./work/tobi/parquet/{name}.parquet")
    df.createOrReplaceTempView(name)
    df.printSchema()
    print("#Rows:", df.count())
    df.show(2)
    print()
    return df

In [3]:
activity_logs = load("activity_logs")
financial_journal = load("financial_journal")

===================== activity_logs =====================
root
 |-- timestamp: timestamp (nullable = true)
 |-- currentLocation: string (nullable = true)
 |-- participantId: integer (nullable = true)
 |-- currentMode: string (nullable = true)
 |-- hungerStatus: string (nullable = true)
 |-- sleepStatus: string (nullable = true)
 |-- apartmentId: integer (nullable = true)
 |-- availableBalance: float (nullable = true)
 |-- jobId: integer (nullable = true)
 |-- financialStatus: string (nullable = true)
 |-- dailyFoodBudget: float (nullable = true)
 |-- weeklyExtraBudget: float (nullable = true)
 |-- currentLocationX: float (nullable = true)
 |-- currentLocationY: float (nullable = true)
 |-- date: date (nullable = true)

#Rows: 113923735
+-------------------+--------------------+-------------+-----------+------------+-----------+-----------+----------------+-----+---------------+---------------+-----------------+----------------+----------------+----------+
|          timestamp|     curr

In [4]:
activity_logs.select('timestamp', 'sleepStatus', 'participantId').where((activity_logs.currentMode == 'AtHome')).show(5)

+-------------------+--------------+-------------+
|          timestamp|   sleepStatus|participantId|
+-------------------+--------------+-------------+
|2022-03-03 00:00:00|      Sleeping|            0|
|2022-03-03 00:00:00|      Sleeping|            2|
|2022-03-03 00:00:00|      Sleeping|            3|
|2022-03-03 00:00:00|      Sleeping|            4|
|2022-03-03 00:00:00|PrepareToSleep|            5|
+-------------------+--------------+-------------+
only showing top 5 rows



In [5]:
spark.sql("SELECT timestamp, sleepStatus, hungerStatus, financialStatus, count(*) from activity_logs group by timestamp, sleepStatus, hungerStatus, financialStatus order by timestamp asc").show()

+-------------------+-----------+------------+---------------+--------+
|          timestamp|sleepStatus|hungerStatus|financialStatus|count(1)|
+-------------------+-----------+------------+---------------+--------+
|2022-03-01 00:00:00|   Sleeping|     JustAte|         Stable|     669|
|2022-03-01 00:00:00|      Awake|     JustAte|         Stable|     196|
|2022-03-01 00:00:00|   Sleeping|     JustAte|        Unknown|     111|
|2022-03-01 00:00:00|      Awake|     JustAte|        Unknown|      31|
|2022-03-01 00:00:00|      Awake|     JustAte|       Unstable|       3|
|2022-03-01 00:00:00|   Sleeping|     JustAte|       Unstable|       1|
|2022-03-01 00:05:00|   Sleeping|     JustAte|       Unstable|       1|
|2022-03-01 00:05:00|      Awake|     JustAte|        Unknown|      29|
|2022-03-01 00:05:00|   Sleeping|     JustAte|        Unknown|     113|
|2022-03-01 00:05:00|      Awake|     JustAte|       Unstable|       3|
|2022-03-01 00:05:00|      Awake|     JustAte|         Stable|  

In [6]:
spark.sql("select category, sum(amount) from financial_journal group by category order by category asc").show()

+--------------+-------------------+
|      category|        sum(amount)|
+--------------+-------------------+
|     Education|-188635.79949443735|
|          Food| -4562517.832099259|
|    Recreation| -4973094.303666381|
|RentAdjustment|  54932.73219610953|
|       Shelter| -8982423.083224718|
|          Wage|5.563302183606546E7|
+--------------+-------------------+



In [7]:
alt.Chart(spark.sql("select category, sum(amount) as total from financial_journal group by category order by total desc").toPandas()).mark_bar().encode(
    alt.Y('category', type='nominal'),
    alt.X('total', type = 'quantitative')
)

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [8]:
print(spark.sql("select distinct date(timestamp) as d from financial_journal").count())

451


In [9]:
df = spark.sql("select date(timestamp) as date, category, sum(amount) as total from financial_journal group by 1, 2 order by 1 asc, 2 asc").toPandas()
df["date"] = pd.to_datetime(df["date"])

alt.Chart(df[df["date"] > "2022-03-01"]).mark_area().encode( # Remove 2022-March-01 as it has very odd values
    y=alt.Y('total', title="Total", scale=alt.Scale(domain=[-5e5, 2e5])), 
    x=alt.X('date', title='Date'), 
    color='category',
    tooltip=['date', 'total', 'category'],
    row = 'category'
).properties(height=100, width=1000, title="Population financial transactions")

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [10]:
total = spark.sql("select distinct date_trunc('day', timestamp), participantId from activity_logs").count()

hunger = spark.sql(f"""
    select to_timestamp(date_format(timestamp, "HH:mm"), "HH:mm") as clocktime, hungerStatus as status, count(*) / {total} as percentage 
    from activity_logs group by 1, 2
""").toPandas() 
sleep = spark.sql(f"""
    select to_timestamp(date_format(timestamp, "HH:mm"), "HH:mm") as clocktime, sleepStatus as status, count(*) / {total} as percentage 
    from activity_logs group by 1, 2
""").toPandas() 
financial = spark.sql(f"""
    select to_timestamp(date_format(timestamp, "HH:mm"), "HH:mm") as clocktime, financialStatus as status, count(*) / {total} as percentage 
    from activity_logs group by 1, 2
""").toPandas() 
location = spark.sql(f"""
    select to_timestamp(date_format(timestamp, "HH:mm"), "HH:mm") as clocktime, currentMode as status, count(*) / {total} as percentage 
    from activity_logs group by 1, 2
""").toPandas() 

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [11]:
hunger.drop(hunger[hunger['status'] == 'NA'].index, inplace=True) # Limpiar 1 registro corrupto
hunger.drop(hunger[hunger['status'] == 'Beco'].index, inplace=True) # Limpiar 1 registro corrupto

sortOrder = ["Starving", "Hungry", "BecomingHungry", "JustAte", "BecameFull"]

alt.Chart(hunger).mark_area().encode(
    x = alt.X('clocktime:T', title="Time of day", axis=alt.Axis(format="%H:%M")),
    y = alt.Y('percentage:Q', title="Population", axis=alt.Axis(format='%')),
    color = alt.Color('status', title="Status", sort=sortOrder, scale=alt.Scale(domain=sortOrder, range=['darkred', 'orange', 'gold', 'steelblue', 'green'])),
    tooltip = [alt.Tooltip('clocktime:T', format="%H:%M"), alt.Tooltip('percentage', format=".2%"), 'status']
).properties(width=1000, title='Hunger status of the population')

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [12]:
location.drop(location[location["status"] == "NA"].index, inplace=True)

alt.Chart(location).mark_area().encode(
    x = alt.X('clocktime:T', title="Time of day", axis=alt.Axis(format="%H:%M")),
    y = alt.Y('percentage:Q', title="Population", axis=alt.Axis(format='%')),
    color = alt.Color('status', title="Location"),
    tooltip = [alt.Tooltip('clocktime:T', format="%H:%M"), alt.Tooltip('percentage', format=".2%"), 'status']
).properties(width=1000, title='Location of the population')

alt.Chart(...)

In [13]:
sleep.drop(sleep[sleep["status"] == "NA"].index, inplace=True) # Limpiar 2 registros corruptos

sortOrder = ["Awake", "PrepareToSleep", "Sleeping"]
alt.Chart(sleep).mark_area().encode(
    x = alt.X('clocktime:T', title="Time of day", axis=alt.Axis(format="%H:%M")),
    y = alt.Y('percentage:Q', title="Population", axis=alt.Axis(format='%')),
    color = alt.Color('status', title="Status", sort=sortOrder, scale = alt.Scale(domain=sortOrder, range=['coral', 'yellow', 'steelblue'])),
    tooltip = [alt.Tooltip('clocktime:T', format="%H:%M"), alt.Tooltip('percentage', format=".2%"), 'status']
).properties(width=1000, title='Sleep status of the population')

alt.Chart(...)

In [14]:
alt.Chart(financial).mark_area().encode(
    x = alt.X('clocktime:T', title="Time of day", axis=alt.Axis(format="%H:%M")),
    y = alt.Y('percentage:Q', title="Population", axis=alt.Axis(format='%')),
    color = alt.Color('status', title="Status", sort=sortOrder),
    tooltip = [alt.Tooltip('clocktime:T', format="%H:%M"), alt.Tooltip('percentage', format=".2%"), 'status'],
).properties(width=1000, title='Financial status of the population')

alt.Chart(...)

In [15]:
travel_journal = load('travel_journal')

===================== travel_journal =====================
root
 |-- participantId: integer (nullable = true)
 |-- travelStartTime: timestamp (nullable = true)
 |-- travelStartLocationId: integer (nullable = true)
 |-- travelEndTime: timestamp (nullable = true)
 |-- travelEndLocationId: integer (nullable = true)
 |-- purpose: string (nullable = true)
 |-- checkInTime: timestamp (nullable = true)
 |-- checkOutTime: timestamp (nullable = true)
 |-- startingBalance: double (nullable = true)
 |-- endingBalance: double (nullable = true)

#Rows: 2099656
+-------------+-------------------+---------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-----------------+------------------+
|participantId|    travelStartTime|travelStartLocationId|      travelEndTime|travelEndLocationId|             purpose|        checkInTime|       checkOutTime|  startingBalance|     endingBalance|
+-------------+-------------------+-------------------

In [16]:
gridSquareSize = 25
position = spark.sql(f"""
    select 
        round(int(currentLocationX) / {gridSquareSize}) * {gridSquareSize} as currentLocationX, 
        round(int(currentLocationY) / {gridSquareSize}) * {gridSquareSize} as currentLocationY,  
        count(*) as frequency 
    from activity_logs group by 1, 2 
""")
position.show()
position.count()

+----------------+----------------+---------+
|currentLocationX|currentLocationY|frequency|
+----------------+----------------+---------+
|         -3800.0|          5150.0|     9128|
|         -3750.0|          5150.0|    63739|
|         -3850.0|          7050.0|      649|
|         -4550.0|          7125.0|      326|
|         -2150.0|          4650.0|     6109|
|         -2525.0|          5850.0|     9655|
|         -3850.0|          6575.0|     6328|
|         -1950.0|          3550.0|     4747|
|         -1950.0|          4850.0|      550|
|          1775.0|          4500.0|     3752|
|          -725.0|          1375.0|    66596|
|           925.0|          4025.0|      751|
|           575.0|          4475.0|     3531|
|         -1625.0|          1025.0|      768|
|         -3075.0|          6050.0|     2203|
|          1425.0|          4525.0|      981|
|         -1925.0|          4325.0|      966|
|          -275.0|          4850.0|    31138|
|         -3825.0|          5150.0

4876

In [17]:
positionPd = position.toPandas()

In [18]:
alt.Chart(positionPd).mark_square(size=50, opacity=0.75).encode(
    x = alt.X('currentLocationX', title = 'X-Coordinate'),
    y = alt.Y('currentLocationY', title = 'Y-Coordinate'),
    color = alt.Color('frequency:Q', title = 'Frequency'),
    tooltip = ['currentLocationX', 'currentLocationY', 'frequency']
).properties(title="Heatmap of people's location", width=1000, height=1000).interactive()

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)